In [1]:
import pandas as pd
import numpy as np
import requests
import glob
import os
from glob import glob
from os import getcwd
import owncloud
import json
import time
import csv

In [2]:
# Функции для БД
from Q475_sql import db_connect
from Q475_sql import query
from Q475_sql import sql_query
from Q475_sql import sql_query1

In [100]:
loginAtlassianjhubAdmin = "adorofeev"
passwordAtlassianjhubAdmin = "$ZM~2(p}^i27Qz'J"
    
#     link AmazonS3 nextcloud client login
amazon_owncloud_client = owncloud.Client('https://nextcloud.e2e4.ru')
#     amazon_owncloud_client = 'https://nextcloud.e2e4.ru'
amazon_owncloud_client.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)

AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
AmazonS3_path_to_file = r'/Проверка на ДУБЛИ ("ID клиента" - Аутсорсер, Обслуживающая компания) [АВТОЗАПУСК] Источники для BI (Справочники).py Клиенты (справочник)/'

AmazonS3_full_path_customer = AmazonS3_folder_Public + AmazonS3_path_to_file + r'Cтатус ЮЛ'

In [4]:
temp_tmp = glob('/tmp', recursive = True)[0] + '/'

In [5]:
sql_query = sql_query()
df_rep = query(sql_query)

1. Соединение с PostgreSQL установлено. Загружаем данные...
2. Заказы успешно выгружены
3. Соединение с PostgreSQL закрыто


In [ ]:
# Выгружаем список кастомеров

sql_query1 = sql_query1()
df_cast = query(sql_query1)
df_ul = df_cast[df_cast['inn'].notna()]

Парсим данные с fek.ru

In [ ]:
link = 'https://fek.ru/search-company?query={}'

def get_json(query):
    #time.sleep(1)
    session = requests.session()
    session.headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
        "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
    }

    res = session.get(link.format(query))
    res.encoding = 'utf8'
    html = res.text
    js = json.loads(html)
    return js

def get_status(js, inn):
    for i in js['suggestions']:
        kod = i['data']['inn']
        if kod == inn:
            status = i['data']['state']['status']
            break
        else:
            status = 'ИНН не найден'
            continue
    return status

def parse(row):
    try:
        json_response = get_json(row)
        if json_response['suggestions']:
            status = get_status(json_response, row)
        else:
            status = 'Статус не получен'+json_response['message']
        return status
    except Exception as e:
        return e

In [ ]:
# Настройки для парсинга

x = 60000 #стартовый индекс
n = 1000 # размер партиции
m = 7 # Количество партиций

name_file = 'parse_fek.ru_part_{}'

In [ ]:
# Запуск парсера!!!

for i in range(8):    
    for _ in range(m):
        df = df_ul.iloc[x:x+n]
        x += n
        df['status'] = df['inn'].apply(parse)
        df.to_csv(temp_tmp + name_file.format(x) + '.csv', sep = ';', index=False)
        
    df_excel = pd.DataFrame()
    ind = x - m*n
    for j in range(ind+n, x+1, n):
        file = temp_tmp + name_file.format(j) + '.csv'    
        d = pd.read_csv(file, sep = ';')
        df_excel = pd.concat([df_excel, d], ignore_index = True)
        os.remove(file)
    df_excel.to_excel(temp_tmp + AmazonS3_full_path_customer.split('/')[-1]+ str(i) + '.xlsx',  index=False, encoding='cp-1251') #   to_excel

df_total = pd.DataFrame()
for k in range(8):
    tab = temp_tmp + AmazonS3_full_path_customer.split('/')[-1]+ str(k) + '.xlsx'
    t = pd.read_excel(tab)
    df_total = pd.concat([df_total, t], ignore_index = True)
    os.remove(tab)
df_total.to_excel(temp_tmp + AmazonS3_full_path_customer.split('/')[-1]+ 'total' + '.xlsx',  index=False, encoding='cp-1251') #   to_excel

In [6]:
total_name = AmazonS3_full_path_customer.split('/')[-1]+ 'total' + '.xlsx'
total_name

#amazon_owncloud_client.put_file( AmazonS3_full_path_customer + '_fek.ru_04.08.23' + '.xlsx', temp_tmp + total_name)

'Парсинг статуса ЮЛtotal.xlsx'

In [106]:
df_rep[df_rep['id компании'] == 5038]

,id компании,Название компании,Головная организация,Филиалы,Аутсорсеры,Статус компании
18,5038,ВТБ 24 (ПАО),302534.0,"[275821, 207106, 164328, 144613, 140993, 97442...",None,ЮЛ активно


In [8]:
# Выводим результат парсинга в датафрейм, оставляем только ликвидированные фирмы

df_total = pd.read_excel(temp_tmp + total_name)
df_liq = df_total[(df_total['status'] == 'LIQUIDATED') | (df_total['status'] == 'LIQUIDATING')]
df_liq['status'] = 'ЮЛ закрыто/ликвидировано'

/tmp/ipykernel_2260594/604300140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liq['status'] = 'ЮЛ закрыто/ликвидировано'


In [113]:
# Объединяем результат парсинга с дубликатами из БД

df_report = df_rep.merge(df_liq, how='outer', left_on=['id компании', 'Название компании'], right_on=['id', 'name'])
df_report.loc[df_report['status'].isna(), 'status'] = df_report['Статус компании']
df_report.loc[df_report['id компании'].isna(), 'id компании'] = df_report['id']
df_report.loc[df_report['Название компании'].isna(), 'Название компании'] = df_report['name']
df_report['id компании'] = df_report['id компании'].astype('int64')
#df_report['Головная организация'] = df_report['Головная организация'].astype('Int64')

In [114]:
# Уточняем шапку таблицы

df_report = df_report[['id компании', 'Название компании', 'Головная организация', 'Филиалы', 'Аутсорсеры', 'status']]
df_report.columns = ['id компании', 'Название компании', 'Головная организация', 'Филиалы', 'Аутсорсеры', 'Статус компании']

In [115]:
# Преобразовываем id компаний в ссылки

def id_to_link(row):
    if pd.notna(row):
        lnk = f'@ГИПЕРССЫЛКА("http://opentech.local/index.php?p=customers&id={int(row)}"; {int(row)})'
    else:
        lnk = np.nan
    return lnk

In [116]:
df_report['id компании'] = df_report['id компании'].apply(id_to_link)
df_report['Головная организация'] = df_report['Головная организация'].apply(id_to_link)

In [117]:
df_report

,id компании,Название компании,Головная организация,Филиалы,Аутсорсеры,Статус компании
0,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО ""Сибирская Канцелярская Компания""",NaN,None,None,ЮЛ закрыто/ликвидировано
1,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ЗАО ""АРКА Текнолоджиз""",NaN,None,None,ЮЛ закрыто/ликвидировано
2,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО ""Система""",NaN,None,None,ЮЛ закрыто/ликвидировано
3,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО ""Дюрон Техногрупп""",NaN,None,None,ЮЛ закрыто/ликвидировано
4,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","АО ""Гемма - 2000""",NaN,None,None,ЮЛ закрыто/ликвидировано
...,...,...,...,...,...,...
26550,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...",ИП Даньшин Денис Сергеевич,NaN,NaN,NaN,ЮЛ закрыто/ликвидировано
26551,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО ""УСОЛЬМАШ""",NaN,NaN,NaN,ЮЛ закрыто/ликвидировано
26552,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО НТЦ ""РМ НАНОТЕХ""",NaN,NaN,NaN,ЮЛ закрыто/ликвидировано
26553,"@ГИПЕРССЫЛКА(""http://opentech.local/index.php?...","ООО ""ЮНАЙТЕД СЕРВИС""",NaN,NaN,NaN,ЮЛ закрыто/ликвидировано


In [118]:
# Выгружаем данные в excel и в облако 

df_report.to_excel(temp_tmp + AmazonS3_full_path_customer.split('/')[-1]+ '_report' + '.xlsx',  index=False, encoding='cp-1251') #   to_excel
amazon_owncloud_client.put_file( AmazonS3_full_path_customer + '_report_links' + '.xlsx', temp_tmp + AmazonS3_full_path_customer.split('/')[-1]+ '_report' + '.xlsx')

True